In [12]:
import sqlalchemy
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM,Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
import numpy as np
from tensorflow.keras.optimizers import Adam
import math
from sklearn.metrics import mean_squared_error

pd.option_context('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#Importation des données de la base MSSQL avec sqlalchemy
server="localhost"
database="GIPA"
driver ="ODBC Driver 17 for SQL Server"
con = f'mssql://@{server}/{database}?driver={driver}'
engine = sqlalchemy.create_engine(con,fast_executemany=True)
con = engine.connect()
sql = "select NumFacture,CodeArticle,QteFacturee,CA from GIPA.dbo.dimension_facture"
df = pd.read_sql(sql,con)  #mise en place des données dans un DataFrame 



In [40]:
df_finale = df.groupby(['NumFacture','CodeArticle'])['QteFacturee'].sum().unstack().reset_index().fillna(0).set_index('NumFacture') #transformer la dataset en format convenable pour l'algorithme apriori

In [47]:
df_finale.applymap(lambda x:abs(x))

CodeArticle,18100301,18100901,18120401,18120901,18121201,18121301,18130001,18150001,9002,9003,...,a2mj025f24,a60000b,a611001,a620000,a655000,m61041,m61148,zz0100100,zz70900030,zz70900050
NumFacture,,,,,,,,,,,,,,,,,,,,,
07D00917,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00918,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00919,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00920,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00921,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RFA00129,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
RFA00130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
RFA00131,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [55]:
df_finale.abs()


CodeArticle,18100301,18100901,18120401,18120901,18121201,18121301,18130001,18150001,9002,9003,...,a2mj025f24,a60000b,a611001,a620000,a655000,m61041,m61148,zz0100100,zz70900030,zz70900050
NumFacture,,,,,,,,,,,,,,,,,,,,,
07D00917,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00918,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00919,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00920,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
07D00921,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RFA00129,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
RFA00130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
RFA00131,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [56]:
df_finale.describe()

CodeArticle,18100301,18100901,18120401,18120901,18121201,18121301,18130001,18150001,9002,9003,...,a2mj025f24,a60000b,a611001,a620000,a655000,m61041,m61148,zz0100100,zz70900030,zz70900050
count,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,...,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000,133432.000
mean,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.002,0.021,23.264,...,0.010,0.004,0.003,0.005,0.002,0.014,0.009,-0.018,-0.001,-0.000
std,0.043,0.033,0.307,0.092,0.092,0.093,0.137,0.297,2.394,121.048,...,0.682,0.157,0.099,0.173,0.070,0.753,0.582,0.134,0.036,0.016
min,-6.000,-6.000,-6.000,-6.000,-6.000,-6.000,-2.000,-6.000,-240.000,-3120.000,...,0.000,0.000,0.000,-20.000,0.000,0.000,-5.000,-1.000,-1.000,-2.000
25%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,10.000,6.000,60.000,30.000,30.000,30.000,38.000,60.000,600.000,3350.000,...,120.000,15.000,5.000,22.000,5.000,86.000,120.000,1.000,1.000,0.000


In [57]:
def hot_encode(resultat):
    if resultat  == 0:
        return 0
    else:
        return 1
        
    

In [68]:
df_finale = df_finale.applymap(lambda x:abs(x))

In [70]:
df_finale = df_finale.applymap(lambda x:hot_encode(x))

In [72]:
from mlxtend.frequent_patterns import apriori,association_rules
article_frequence = apriori(df_finale,min_support=0.07,use_colnames=True)


In [80]:
article_frequence

,support,itemsets
0,0.176,(9003)
1,0.070,(9070)
2,0.085,(9071)
3,0.226,(9146)
4,0.098,(92391)
...,...,...
89,0.070,"(93394, 9247, 9146)"
90,0.086,"(93394, 93310, 9146)"
91,0.072,"(9360, 93310, 9146)"
92,0.084,"(93395, 93394, 9146)"


In [81]:
regles = association_rules(article_frequence,metric='lift',min_threshold=1)
 

In [93]:
regles



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(9247),(9146),0.131,0.226,0.097,0.737,3.261,0.067,2.945
1,(9146),(9247),0.226,0.131,0.097,0.428,3.261,0.067,1.519
2,(9265),(9146),0.110,0.226,0.089,0.805,3.562,0.064,3.973
3,(9146),(9265),0.226,0.110,0.089,0.393,3.562,0.064,1.465
4,(93310),(9146),0.156,0.226,0.119,0.761,3.368,0.084,3.244
...,...,...,...,...,...,...,...,...,...
109,"(9360, 9146)",(93394),0.112,0.186,0.082,0.728,3.903,0.061,2.987
110,"(93394, 9146)",(9360),0.139,0.151,0.082,0.586,3.896,0.061,2.054
111,(9360),"(93394, 9146)",0.151,0.139,0.082,0.542,3.896,0.061,1.879
112,(93394),"(9360, 9146)",0.186,0.112,0.082,0.437,3.903,0.061,1.578
